<a href="https://colab.research.google.com/github/Abeszz/SC4002-NLP-Assignment/blob/main/SC4002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Requirements :

In [1]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00


In [2]:
pip install nltk

In [3]:
pip install numpy

In [4]:
pip install npm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for npm: filename=npm-0.1.1-py3-none-any.whl size=3683 sha256=380fd610d537f6261352c250969f46fcc8927db758a22f0734bc161eaa03be19
  Stored in directory: /root/.cache/pip/wheels/03/5c/79/747f074a7b810b0044b9a05fe3e3c04ed088074577b391eb0c
  Created wheel for optional-django: filename=optional_django-0.1.0-py3-none-any.whl size=9950 sha256=05ec19cffdc0152f3b29defc892395c591b21c7a98461b1d48325eeb50c81fec
  Stored in directory: /root/.cache/pip/wheels/24/47/0a/6bdc915164c9c29e3a7d93ea1ef469c1079e2f427bce820d43
Successfully built npm optional-django


In [5]:
pip install torch

In [6]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-10-19 06:08:53--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-19 06:08:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  3.20MB/s    in 3m 26s  

2024-10-19 06:12:19 (3.98 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [8]:
import os
import nltk
import sys

env_base_path = sys.prefix
nltk_path = os.path.join(env_base_path, 'nltk_data')
nltk.download('punkt', nltk_path)
nltk.download('punkt_tab', nltk_path)

[nltk_data] Downloading package punkt to /usr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /usr/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **PART 0. Dataset Preparation**

In [9]:
from datasets import load_dataset
dataset = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

# **Part 1. Preparing Word Embedding**

In [11]:
vocabulary = None
vocab_size = None
embedding_matrix = None
oov_words = [] # Array to track OOV words
UNKNOWN_TOKEN = '<UNKNOWN>'  # Special token for OOV words

glove_model_file = 'glove.6B.300d.txt'
glove_model_dimension = 300

In [12]:
from nltk.tokenize import word_tokenize
from collections import Counter

def tokenize_sentence(sentence):
    # Tokenize each sentence using NLTK's word_tokenize
    return word_tokenize(sentence.lower()) # Convert to lowercase for consistency

def tokenize_dataset(dataset):
    global vocabulary, vocab_size
    # Build a vocabulary from the dataset
    vocab_counter = Counter()

    for word in dataset:
        tokens = tokenize_sentence(word['text'])
        vocab_counter.update(tokens)

    # Update the vocabulary list
    vocabulary = list(vocab_counter.keys())
    vocab_size = len(vocabulary)

In [13]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

In [14]:
import numpy as np

def create_embedding_matrix(glove_file_path, glove_model_dimension):
    global vocabulary, embedding_matrix, oov_words
    # Load the GloVe embeddings into a dictionary
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create the embedding matrix
    embedding_matrix = np.zeros((vocab_size, glove_model_dimension))  # Initialize the matrix with zeros

    # Initialize the UNKNOWN token with a random vector
    unknown_vector = np.random.normal(scale=0.6, size=(glove_model_dimension,))

    # Fill the embedding matrix
    for idx, word in enumerate(vocabulary):
        if word in glove_embeddings:
            embedding_matrix[idx] = glove_embeddings[word]  # Use the pretrained GloVe vector
        elif word == UNKNOWN_TOKEN:
            embedding_matrix[idx] = unknown_vector  # Do not add UNKNOWN token to oov
        else:
            oov_words.append(word)  # Track OOV words
            embedding_matrix[idx] = unknown_vector

In [15]:
def add_unkown_token():
    global vocabulary, vocab_size, UNKNOWN_TOKEN

    # Check if 'UNK' is already in the vocabulary, if not, add it
    if UNKNOWN_TOKEN not in vocabulary:
        vocabulary.append(UNKNOWN_TOKEN)
        vocab_size += 1


In [16]:
# Tokenize the dataset
tokenize_dataset(train_dataset)

In [17]:
# Print the size of Vocabulary
print(f"1(a) Number of Vocabulary words in training dataset = {vocab_size}")

1(a) Number of Vocabulary words in training dataset = 18029


In [18]:
# 1(c) : Add <UNKNOWN> token and replace all OOV words with it
add_unkown_token()

In [19]:
# Initialize the glove model with our configuration
create_embedding_matrix(glove_model_file, glove_model_dimension)

In [20]:
# Print the number of OOV words
print(f"1(b) Number of Out Of Vocabulary words: {len(oov_words)}")

1(b) Number of Out Of Vocabulary words: 1865


In [82]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from nltk.tokenize import word_tokenize
import json
import csv

In [83]:
# Custom collate_fn to handle padding
def collate_fn(batch):
    inputs, labels = zip(*batch)
    inputs = [torch.tensor(x) for x in inputs]
    padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)  # Pad sequences
    labels = torch.stack(labels)
    return padded_inputs, labels

In [84]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, rnn_type="RNN", num_layers=1, use_bidirectional=False, use_dropout=False, use_batch_norm=False, use_layer_norm=False):
        super(SentimentRNN, self).__init__()

        vocab_size, embedding_dim = embedding_matrix.shape

        # Embedding layer using pre-trained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False  # freeze embeddings

        # Choose RNN type dynamically
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=use_bidirectional)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=use_bidirectional)
        else:  # Default to Simple RNN
            self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=use_bidirectional)

        # Fully connected layer for classification
        if use_bidirectional:
            self.fc = nn.Linear(hidden_size * 2, output_size)  # Double hidden size for bidirectional RNN
        else:
            self.fc = nn.Linear(hidden_size, output_size)

        # Optional Regularization Layers
        self.use_dropout = use_dropout
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm

        if self.use_dropout:
            self.dropout = nn.Dropout(0.3)  # Dropout rate of 0.3

        if self.use_batch_norm:
            self.batch_norm = nn.BatchNorm1d(hidden_size * (2 if use_bidirectional else 1))

        if self.use_layer_norm:
            self.layer_norm = nn.LayerNorm(hidden_size * (2 if use_bidirectional else 1))

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)

        # For LSTM, hidden contains two values (h_n, c_n), so we only use h_n
        if isinstance(hidden, tuple):
            hidden = hidden[0]  # Use h_n

        # Use the last hidden state for uni-directional RNN, or concatenate the last forward and backward hidden states for bidirectional
        if self.rnn.bidirectional:
            final_output = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            final_output = hidden[-1]

        # Apply optional regularization layers
        if self.use_batch_norm:
            final_output = self.batch_norm(final_output)

        if self.use_layer_norm:
            final_output = self.layer_norm(final_output)

        if self.use_dropout:
            final_output = self.dropout(final_output)

        return self.fc(final_output)


In [85]:
# Dataset class to handle data batching
class TextDataset(Dataset):
    def __init__(self, dataset, vocab):
        self.dataset = dataset
        self.vocab = vocab

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sentence = self.dataset[idx]['text']
        label = self.dataset[idx]['label']
        tokens = word_tokenize(sentence.lower())
        indices = [self.vocab.index(token) if token in self.vocab else self.vocab.index("<UNKNOWN>") for token in tokens]
        return torch.tensor(indices), torch.tensor(label)

In [86]:
def get_optimizer(params, model):
    optimizer_type = params["optimizer_type"]
    lr = params["learning_rate"]
    weight_decay = params.get("weight_decay", 0)  # Default to 0 if not specified

    if optimizer_type == "SGD":
        momentum = params.get("momentum", 0)  # Default to 0 if not specified
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    elif optimizer_type == "Adam":
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    else:
        raise ValueError(f"Unknown optimizer type: {optimizer_type}")

In [89]:
# def train_model(train_dataset, validation_dataset, vocabulary, embedding_matrix, params_file="hyperparams.txt", optimizer_file="optimizer_params.txt"):
#     # Read hyperparameters from file
#     with open(params_file, "r") as f:
#         params = json.load(f)

#     # Load optimizer parameters
#     with open(optimizer_file, "r") as f:
#         optimizer_params = json.load(f)

#     output_size = params['output_size']
#     hidden_size = params['hidden_size']
#     batch_size = params['batch_size']
#     epochs = params['epochs']
#     lr = params['learning_rate']
#     patience = params['patience']

#     model = SentimentRNN(embedding_matrix, hidden_size, output_size)

#     # Get the optimizer dynamically based on the configuration in optimizer_file
#     optimizer = get_optimizer(optimizer_params, model)

#     criterion = nn.CrossEntropyLoss()
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#     # Create DataLoaders for training and validation with custom collate_fn for padding
#     train_loader = DataLoader(TextDataset(train_dataset, vocabulary), batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
#     valid_loader = DataLoader(TextDataset(validation_dataset, vocabulary), batch_size=batch_size, collate_fn=collate_fn)

#     best_val_accuracy = 0
#     epochs_no_improve = 0

#     # Prepare for CSV logging
#     csv_filename = 'training_output.csv'
#     fieldnames = ['Epoch', 'Train Loss', 'Validation Accuracy', 'Test Accuracy', 'output_size', 'hidden_size', 'batch_size', 'epochs', 'learning_rate', 'patience', 'optimizer_type', 'momentum', 'weight_decay']

#     # Combine optimizer and hyperparameters into a single dictionary for consistency
#     optimizer_params_combined = {**params, **optimizer_params}

#     # Add parameters to CSV as well
#     with open(csv_filename, mode='w', newline='') as file:
#         writer = csv.DictWriter(file, fieldnames=fieldnames)
#         writer.writeheader()

#         for epoch in range(epochs):
#             print(f"Starting Epoch {epoch + 1}/{epochs}")
#             model.train()
#             running_loss = 0.0
#             for inputs, labels in train_loader:
#                 optimizer.zero_grad()
#                 outputs = model(inputs)
#                 loss = criterion(outputs, labels)
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item()

#             train_loss = running_loss / len(train_loader)
#             print(f'Epoch {epoch+1}, Loss: {train_loss}')

#             # Validation
#             model.eval()
#             correct = 0
#             total = 0
#             with torch.no_grad():
#                 for inputs, labels in valid_loader:
#                     outputs = model(inputs)
#                     _, predicted = torch.max(outputs.data, 1)
#                     total += labels.size(0)
#                     correct += (predicted == labels).sum().item()

#             val_accuracy = 100 * correct / total
#             print(f'Validation Accuracy: {val_accuracy}%')

#             # Evaluate model on the test set after each epoch
#             test_accuracy = evaluate_model_on_test(model, test_dataset, vocabulary)

#             # Log results to CSV
#             with open(csv_filename, mode='a', newline='') as file:
#                 writer = csv.DictWriter(file, fieldnames=fieldnames)
#                 writer.writerow({
#                     'Epoch': epoch + 1,
#                     'Train Loss': train_loss,
#                     'Validation Accuracy': val_accuracy,
#                     'Test Accuracy': test_accuracy,
#                     'output_size': params['output_size'],
#                     'hidden_size': params['hidden_size'],
#                     'batch_size': params['batch_size'],
#                     'epochs': params['epochs'],
#                     'learning_rate': params['learning_rate'],
#                     'patience': params['patience'],
#                     'optimizer_type': optimizer_params.get('optimizer_type', 'SGD'),
#                     'momentum': optimizer_params.get('momentum', 0),
#                     'weight_decay': optimizer_params.get('weight_decay', 0)
#                 })

#             # Early stopping logic: check if validation accuracy improved
#             if val_accuracy > best_val_accuracy:
#                 best_val_accuracy = val_accuracy
#                 epochs_no_improve = 0  # Reset the patience counter
#                 torch.save(model.state_dict(), 'best_model.pt')  # Save the best model's weights
#             else:
#                 epochs_no_improve += 1

#             scheduler.step()  # Step the learning rate scheduler

#             if epochs_no_improve >= patience:
#                 print(f'Early stopping at epoch {epoch+1}')
#                 break

#             print("Evaluating model on test set...")
#             test_accuracy = evaluate_model_on_test(model, test_dataset, vocabulary)

#     print("Training completed. Results saved to CSV.")
#     return model


In [94]:
def train_model(model, train_loader, valid_loader, test_dataset, vocabulary, optimizer, epochs, patience, scheduler_step_size=3, scheduler_gamma=0.1):
    """
    Train the RNN model with the given parameters.

    Args:
    - model: The initialized SentimentRNN model.
    - train_loader: DataLoader for the training dataset.
    - valid_loader: DataLoader for the validation dataset.
    - test_dataset: The test dataset.
    - vocabulary: Vocabulary used for tokenization.
    - optimizer: Optimizer for training.
    - epochs: Number of epochs for training.
    - patience: Early stopping patience value.
    - scheduler_step_size: The step size for the learning rate scheduler.
    - scheduler_gamma: The decay factor for the learning rate scheduler.

    Returns:
    - train_loss_list: List of training losses per epoch.
    - val_accuracy_list: List of validation accuracies per epoch.
    - test_accuracy_list: List of test accuracies per epoch.
    """
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

    best_val_accuracy = 0
    epochs_no_improve = 0
    train_loss_list = []
    val_accuracy_list = []
    test_accuracy_list = []

    for epoch in range(epochs):
        print(f"Starting Epoch {epoch + 1}/{epochs}")
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1}, Loss: {avg_train_loss}')
        train_loss_list.append(avg_train_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Validation Accuracy: {val_accuracy}%')
        val_accuracy_list.append(val_accuracy)

        # Test Accuracy
        test_accuracy = evaluate_model_on_test(model, test_dataset, vocabulary)
        print(f'Test Accuracy: {test_accuracy}%')
        test_accuracy_list.append(test_accuracy)

        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pt')  # Save best model's weights
        else:
            epochs_no_improve += 1

        scheduler.step()

        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break

    return train_loss_list, val_accuracy_list, test_accuracy_list


In [90]:
def evaluate_model_on_test(model, test_dataset, vocabulary):
    # Create a DataLoader for the test dataset
    test_loader = DataLoader(TextDataset(test_dataset, vocabulary), batch_size=32, collate_fn=collate_fn)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and print the test accuracy
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy: {test_accuracy}%')
    return test_accuracy

In [95]:
def run_experiments_from_csv(params_csv, train_dataset, validation_dataset, test_dataset, vocabulary, embedding_matrix):
    csv_filename = 'training_results.csv'
    fieldnames = ['Epoch', 'Train Loss', 'Validation Accuracy', 'Test Accuracy', 'rnn_type', 'num_layers', 'bidirectional', 'dropout', 'batch_norm', 'layer_norm',
                  'optimizer_type', 'learning_rate', 'momentum', 'weight_decay', 'batch_size', 'epochs', 'patience', 'hidden_size', 'output_size']

    # Open the CSV file with the configurations
    with open(params_csv, mode='r') as params_file:
        reader = csv.DictReader(params_file)

        # Prepare the results CSV to store the results for each run
        with open(csv_filename, mode='w', newline='') as result_file:
            writer = csv.DictWriter(result_file, fieldnames=fieldnames)
            writer.writeheader()

            for config in reader:
                # Convert necessary parameters to appropriate types
                config['num_layers'] = int(config['num_layers'])
                config['use_bidirectional'] = config['use_bidirectional'] == 'True'
                config['use_dropout'] = config['use_dropout'] == 'True'
                config['use_batch_norm'] = config['use_batch_norm'] == 'True'
                config['use_layer_norm'] = config['use_layer_norm'] == 'True'
                config['batch_size'] = int(config['batch_size'])
                config['epochs'] = int(config['epochs'])
                config['patience'] = int(config['patience'])
                config['hidden_size'] = int(config['hidden_size'])
                config['output_size'] = int(config['output_size'])
                config['learning_rate'] = float(config['learning_rate'])
                config['momentum'] = float(config['momentum'])
                config['weight_decay'] = float(config['weight_decay'])

                # Initialize the SentimentRNN model with the current config
                model = SentimentRNN(
                    embedding_matrix=embedding_matrix,
                    hidden_size=config['hidden_size'],
                    output_size=config['output_size'],
                    rnn_type=config['rnn_type'],
                    num_layers=config['num_layers'],
                    use_bidirectional=config['use_bidirectional'],
                    use_dropout=config['use_dropout'],
                    use_batch_norm=config['use_batch_norm'],
                    use_layer_norm=config['use_layer_norm']
                )

                # Get the optimizer dynamically based on the config
                optimizer_params = {
                    'optimizer_type': config['optimizer_type'],
                    'learning_rate': config['learning_rate'],
                    'momentum': config['momentum'],
                    'weight_decay': config['weight_decay']
                }
                optimizer = get_optimizer(optimizer_params, model)

                # Train the model using train_model function
                train_loader = DataLoader(TextDataset(train_dataset, vocabulary), batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
                valid_loader = DataLoader(TextDataset(validation_dataset, vocabulary), batch_size=config['batch_size'], collate_fn=collate_fn)

                train_loss, val_accuracy, test_accuracy = train_model(
                    model=model,
                    train_loader=train_loader,
                    valid_loader=valid_loader,
                    test_dataset=test_dataset,
                    vocabulary=vocabulary,
                    optimizer=optimizer,
                    epochs=config['epochs'],
                    patience=config['patience'],
                    scheduler_step_size=3,
                    scheduler_gamma=0.1
                )

                # Log results to CSV after training completes
                for epoch in range(len(train_loss)):  # Iterate through each epoch's results
                    writer.writerow({
                        'Epoch': epoch + 1,
                        'Train Loss': train_loss[epoch],
                        'Validation Accuracy': val_accuracy[epoch],
                        'Test Accuracy': test_accuracy[epoch],
                        'rnn_type': config['rnn_type'],
                        'num_layers': config['num_layers'],
                        'bidirectional': config['use_bidirectional'],
                        'dropout': config['use_dropout'],
                        'batch_norm': config['use_batch_norm'],
                        'layer_norm': config['use_layer_norm'],
                        'optimizer_type': config['optimizer_type'],
                        'learning_rate': config['learning_rate'],
                        'momentum': config['momentum'],
                        'weight_decay': config['weight_decay'],
                        'batch_size': config['batch_size'],
                        'epochs': config['epochs'],
                        'patience': config['patience'],
                        'hidden_size': config['hidden_size'],
                        'output_size': config['output_size']
                    })

    print(f"Experiments completed. Results saved to {csv_filename}.")

In [98]:
params_file = "hyperparams.txt"
optimizer_file = "optimizer_params.txt"
params_csv = "experiment_params1.csv"
# model = train_model(train_dataset, validation_dataset, vocabulary, embedding_matrix, params_file=params_file, optimizer_file=optimizer_file)
model = run_experiments_from_csv(params_csv, train_dataset, validation_dataset, test_dataset, vocabulary, embedding_matrix)


<ipython-input-83-7477fef6ada8>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(x) for x in inputs]


Test Accuracy: 49.15572232645403%
Test Accuracy: 49.34333958724203%
Test Accuracy: 50.093808630393994%
Test Accuracy: 50.187617260787995%
Test Accuracy: 49.34333958724203%
Test Accuracy: 50.0%
Test Accuracy: 50.0%
Early stopping at epoch 7
Experiments completed. Results saved to training_results.csv.
